### Uploading a video and trascribing/summarizing it

In [21]:
import whisper
from openai import OpenAI
import ffmpeg

def transcribe_video(video_path):

    model = whisper.load_model("base")
    
    print("Transcribing the video...")

    result = model.transcribe(video_path)
    
    return result['text']



def summarize_text(transcription_text, client):
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are an assistant that summarizes long texts."},
        {"role": "user", "content": f"Summarize the following content: {transcription_text}. Give 20 key bullet points."}
    ]
    )

    summary = response.choices[0].message.content 
    return summary



if __name__ == "__main__":

    video_path = r"C:\Users\alokd\Downloads\Data Science, Predictive Analytics, and AI in Drug Discovery with Novartis (CXOTalk #717).mp4"

    client = OpenAI(
    api_key="API_KEY_HERE"
    ) 

    #transcription

    transcription = transcribe_video(video_path)

    print("\n--- Transcription ---\n")

    print(transcription)
    

    #summary
    summary = summarize_text(transcription, client)

    print("\n--- Summary ---\n")

    print(summary)

c:\Users\alokd\miniconda3\lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


Transcribing the video...


c:\Users\alokd\miniconda3\lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



--- Transcription ---

 So you've got all of these people working together from different fields. How do you stop them from killing each other? Sometimes you sacrifice yourself as the leader, I would say. Today we are talking about AI and data science in Pharma. And we're speaking with Boulette Cazilton, who's with no Vardis, one of the largest pharmaceutical companies in the world. Yes, my title is the head of causal and predictive analytics at Novartis. Our organization is called the AI Innovation Lab. And what we do is we drive the internal AI innovation across Novartis and also position ourselves in the intersection of academia, tech and the business units where we think AI innovation will really happen. And this is what we do. We are trying to really build an operational excellence with diverse talent coming on board, engaging the domain experts from drug development and discovery. So we can really re-invent some of the processes, speed them up, make them low cost as possible bec

In [24]:
#Keeping summarizing seperate

def summarize_text(transcription_text, client):
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are an assistant that summarizes long texts."},
        {"role": "user", "content": f"Summarize the following content: {transcription_text}. Give 10 key bullet points for perspective of a person trying to apply in role of data science."}
    ]
    )

    summary = response.choices[0].message.content 
    return summary



summary = summarize_text(transcription, client)

print("\n--- Summary ---\n")

print(summary)


--- Summary ---

1. Data science in Pharma involves driving internal AI innovation, focusing on accelerating drug discovery and development processes.
2. The AI Innovation Lab at Novartis aims to reinvent processes, speed them up, and make them cost-effective by leveraging AI and data science.
3. The team focuses on predictive analytics, causal learning, and leveraging both small and big data to extract valuable insights.
4. Strategies are use case-driven and involve collaborating at the intersection of academia, technology, and business units.
5. AI and data science play a crucial role in precision medicine, drug discovery, and generative chemistry.
6. The team works closely with domain experts to understand clinical questions and address biases in data for accurate predictions.
7. Building an operational excellence with diverse talent is key, as different perspectives drive innovation.
8. Challenges include overcoming cultural barriers, engaging different business units, and address

### Using youtube link directly to summarize (using yt_dlp)

In [6]:
import os
import whisper
import yt_dlp
from openai import OpenAI

def download_audio(url, output_path="audio"):
    try:
        # Create output directory if it doesn't exist
        if not os.path.exists(output_path):
            os.makedirs(output_path)
        
        # Configure yt-dlp options
        ydl_opts = {
            'format': 'm4a/bestaudio/best',  # Choose best audio format
            'paths': {'home': output_path},   # Download directory
            'outtmpl': {'default': '%(title)s.%(ext)s'},  # Output template
            'postprocessors': [{  # Convert to mp3
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
            }],
            'quiet': False,
            'no_warnings': True
        }
        
        # Download audio
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            print("Downloading audio...")
            info = ydl.extract_info(url, download=True)
            audio_file = os.path.join(output_path, f"{info['title']}.mp3")
            return audio_file
            
    except Exception as e:
        print(f"Error downloading audio: {str(e)}")
        return None

def transcribe_audio(audio_path, model_size="base"):
    try:
        print("Loading Whisper model...")
        model = whisper.load_model(model_size)
        
        print("Starting transcription...")
        result = model.transcribe(audio_path)
        
        return result["text"]
        
    except Exception as e:
        print(f"Error during transcription: {str(e)}")
        return None

def summarize_text(transcription_text, openai_api_key):
    try:
        client = OpenAI(api_key=openai_api_key)
        
        print("Generating summary...")
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are an assistant that summarizes long texts."},
                {"role": "user", "content": f"Summarize the following content: {transcription_text}. Give 10 very important bullet points"}
            ]
        )
        
        summary = response.choices[0].message.content
        return summary
        
    except Exception as e:
        print(f"Error generating summary: {str(e)}")
        return None

def process_video(url, openai_api_key=None, model_size="base", output_path="audio", save_audio=False):
    """
    Main function to process video: download, transcribe, and optionally summarize
    """
    # Download and transcribe
    audio_file = download_audio(url, output_path)

    if not audio_file:
        return None, None

    transcription = transcribe_audio(audio_file, model_size)

    if not transcription:
        return None, None

    # Summarize if API key provided
    summary = None
    if openai_api_key:
        summary = summarize_text(transcription, openai_api_key)

    # Clean up audio file if not saving
    if not save_audio:
        try:
            os.remove(audio_file)
            print("Audio file deleted")
        except:
            print("Warning: Could not delete audio file")
    else:
        print(f"Audio file saved: {audio_file}")

    return transcription, summary

def main():
    print("YouTube Video Transcriber and Summarizer")
    print("-" * 40)
    
    # Get inputs
    video_url = input("Enter YouTube URL: ").strip()
    if not video_url:
        print("Error: URL cannot be empty")
        return

    openai_api_key = input("Enter OpenAI API key (press Enter to skip summarization): ").strip() or None
    
    save_audio = input("Save audio file? (y/n): ").lower().startswith('y')

    # Process video
    transcription, summary = process_video(
        video_url,
        openai_api_key,
        save_audio=save_audio
    )

    # Handle results
    if transcription:
        print("\nTranscription:")
        print(transcription)
        
        # Save transcription
        with open("transcription.txt", "w", encoding="utf-8") as f:
            f.write(transcription)
        print("\nTranscription saved to transcription.txt")

        # Handle summary if available
        if summary:
            print("\nSummary:")
            print(summary)
            
            with open("summary.txt", "w", encoding="utf-8") as f:
                f.write(summary)
            print("\nSummary saved to summary.txt")

if __name__ == "__main__":
    main()

YouTube Video Transcriber and Summarizer
----------------------------------------
[youtube] Extracting URL: https://www.youtube.com/watch?v=vmAgDc2iY9Q
[youtube] vmAgDc2iY9Q: Downloading webpage
[youtube] vmAgDc2iY9Q: Downloading tv player API JSON
[youtube] vmAgDc2iY9Q: Downloading ios player API JSON
[youtube] vmAgDc2iY9Q: Downloading m3u8 information
[info] vmAgDc2iY9Q: Downloading 1 format(s): 140
[download] Destination: audio\Trump announces investment in AI infrastructure.m4a
[download] 100% of   58.88MiB in 00:00:04 at 14.66MiB/s    
[FixupM4a] Correcting container of "audio\Trump announces investment in AI infrastructure.m4a"
[ExtractAudio] Destination: audio\Trump announces investment in AI infrastructure.mp3
Deleting original file audio\Trump announces investment in AI infrastructure.m4a (pass -k to keep)
Loading Whisper model...


c:\Users\alokd\miniconda3\lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


Starting transcription...


c:\Users\alokd\miniconda3\lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Generating summary...
Audio file deleted

Transcription:
 1 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 個 

###  Creating Telegram BOT for taking YouTube link as input from users and giving output

In [25]:
import os
import asyncio
import nest_asyncio
import whisper
import yt_dlp
from openai import OpenAI
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes
import re

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

# Constants
TELEGRAM_TOKEN = "token here"
OPENAI_API_KEY = "api_key_here"


def extract_video_id(url):
    patterns = [
        r'(?:youtube\.com/watch\?v=|youtu\.be/)([^&\n?]*)',
        r'(?:youtube\.com/shorts/)([^&\n?]*)',
        r'(?:youtube\.com/embed/)([^&\n?]*)'
    ]
    
    for pattern in patterns:
        match = re.search(pattern, url)
        if match:
            return match.group(1)
    return None

#download audio with error handling capacity

def download_audio(url, output_path="audio"):
    try:
        # Create output directory if it doesn't exist
        if not os.path.exists(output_path):
            os.makedirs(output_path)

        video_id = extract_video_id(url)

        if not video_id:
            return None, "Invalid YouTube URL format"

        # Configure yt-dlp options with enhanced settings
        ydl_opts = {
            'format': 'm4a/bestaudio/best',
            'paths': {'home': output_path},
            'outtmpl': {'default': f'{video_id}.%(ext)s'},
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
            }],
            'quiet': True,
            'no_warnings': True,
            'extract_audio': True,
            'audio_format': 'mp3',
            'audio_quality': 0,
            'nocheckcertificate': True,
            'ignoreerrors': False,
            'no_color': True,
            'geo_bypass': True,
            'cookies-from-browser': 'chrome'  # Use cookies from Chrome for age-restricted videos
        }
        
        # Download audio
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            try:
                # First try to extract info
                info = ydl.extract_info(url, download=False)
                if info is None:
                    return None, "Could not access video info"
                
                # Check duration (skip if longer than 30 minutes)
                duration = info.get('duration', 0)
                if duration and duration > 1800:  # 30 minutes
                    return None, "Video is too long (maximum 30 minutes)"
                
                # Download if checks pass
                info = ydl.extract_info(url, download=True)
                audio_file = os.path.join(output_path, f"{video_id}.mp3")
                
                if not os.path.exists(audio_file):
                    return None, "Failed to download audio"
                
                return audio_file, None
                
            except yt_dlp.utils.DownloadError as e:
                error_message = str(e).lower()
                if "private video" in error_message:
                    return None, "This video is private"
                elif "age-restricted" in error_message:
                    return None, "This video is age-restricted"
                elif "removed by the uploader" in error_message:
                    return None, "This video has been removed"
                elif "copyright grounds" in error_message:
                    return None, "This video is not available due to copyright restrictions"
                else:
                    return None, f"Download error: {str(e)}"
                
    except Exception as e:
        return None, f"Error downloading audio: {str(e)}"

def transcribe_audio(audio_path, model_size="base"):
    try:
        if not os.path.exists(audio_path):
            return None, "Audio file not found"
            
        # Check file size (skip if larger than 100MB)
        if os.path.getsize(audio_path) > 100 * 1024 * 1024:
            return None, "Audio file too large"
            
        model = whisper.load_model(model_size)

        result = model.transcribe(
            audio_path,
            language=None,  
            task="transcribe",
            fp16=False  # Use fp32 for better compatibility
        )
        
        if not result or not result.get("text"):
            return None, "No speech detected in the video"
            
        return result["text"], None
        
    except Exception as e:
        return None, f"Error during transcription: {str(e)}"

async def process_video(url):
    """
    Process video with enhanced error handling
    """
    try:
        # Download audio
        audio_file, download_error = download_audio(url)
        if download_error:
            return f"❌ {download_error}"

        # Transcribe audio
        transcription, transcribe_error = transcribe_audio(audio_file)
        if transcribe_error:
            return f"❌ {transcribe_error}"

        # Summarize
        try:
            client = OpenAI(api_key=OPENAI_API_KEY)
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are an assistant that summarizes long texts."},
                    {"role": "user", "content": f"Summarize the following content: {transcription}. Give 10 very important bullet points"}
                ]
            )
            summary = response.choices[0].message.content
        except Exception as e:
            summary = f"Could not generate summary: {str(e)}"

        # Clean up audio file
        try:
            os.remove(audio_file)
        except:
            pass

        return f"📌 Summary:\n\n{summary}"

    except Exception as e:
        return f"❌ Error processing video: {str(e)}"


async def handle_url(update: Update, context: ContextTypes.DEFAULT_TYPE):
  
    text = update.message.text
    
    # Check for valid YouTube URL
    if not extract_video_id(text):
        await update.message.reply_text(

            "\n Please send a valid YouTube URL only.\n"

            "\n 🚨 Short video = faster processing 🚨 \n"
        )
        return

    # Send processing message
    processing_message = await update.message.reply_text(
        "🔄 Processing your video...\n"
        "⏳ This might take a few minutes depending on the video length.\n"
        "Please be patient!"
    )

    # Process the video
    result = await process_video(text)

    # Clean up processing message
    try:
        await processing_message.delete()
    except:
        pass
    
    # Split message if it's too long
    if len(result) > 4096:
        for i in range(0, len(result), 4096):
            await update.message.reply_text(result[i:i+4096])
    else:
        await update.message.reply_text(result)

def run_bot():

    try:
        # Create application
        application = Application.builder().token(TELEGRAM_TOKEN).build()

        # Add handlers
        application.add_handler(CommandHandler("start", start))
        application.add_handler(CommandHandler("help", help_command))
        application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_url))

        # Start the bot
        print("Bot is running...")
        application.run_polling(allowed_updates=Update.ALL_TYPES)

    except Exception as e:
        print(f"Error running bot: {str(e)}")

if __name__ == "__main__":
    run_bot()

Bot is running...
[download] 100% of    2.85MiB in 00:00:00 at 8.34MiB/s:00

c:\Users\alokd\miniconda3\lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


[download] 100% of    7.16MiB in 00:00:00 at 11.96MiB/s00wn

c:\Users\alokd\miniconda3\lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


Error running bot: Cannot close a running event loop
